In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from catboost import Pool, CatBoostClassifier, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
dataset = pd.read_csv("Hu_sk_dataset.csv")
dataset.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,71,72,73,74,75,76,77,78,79,80
0,0,0.png,416.027405,212.334763,0.911107,424.742737,206.524521,0.949083,418.932495,203.619400,...,467.985413,0.891612,468.319580,479.605896,0.785333,453.793945,470.890503,0.890539,3.076171,7
1,1,10.png,411.591034,213.453430,0.948837,417.534088,204.538834,0.911991,414.562561,204.538834,...,469.005341,0.848405,471.021698,477.919952,0.764094,453.192505,471.976868,0.892658,3.019633,7
2,2,20.png,409.421112,215.449310,0.951167,415.245483,206.712738,0.933428,409.421112,206.712738,...,465.897614,0.894365,467.664886,480.458557,0.790847,453.103943,471.721985,0.850276,3.105330,7
3,3,30.png,407.601929,215.433823,0.964128,416.332794,206.702942,0.942259,407.601929,206.702942,...,468.629578,0.867242,468.718140,480.270782,0.774923,451.256348,471.539886,0.880984,3.042257,7
4,4,40.png,404.583527,216.670563,0.935175,410.359406,208.006729,0.931781,404.583527,208.006729,...,467.921509,0.897768,471.006195,479.473267,0.732927,450.790588,470.809448,0.899404,3.013678,7


In [6]:
dataset = dataset.drop(["Unnamed: 0"],axis = 1)

In [7]:
y= dataset['80']
X= dataset.drop(['80','0'], axis=1)
testing=dataset.copy()
testing=testing.drop(['0'], axis=1)
train, test = train_test_split(testing, test_size=0.2)
temp_df = train.groupby(['80']).size().reset_index().rename(columns={0:'count'})
total_count = np.sum(temp_df['count'].values)
temp_df['class%'] = (temp_df['count'] / total_count) * 100
lowest_pct = min(temp_df['class%'])
temp_df['class_weight'] = lowest_pct / temp_df['class%']
class_weight = temp_df[['80', 'class_weight']].to_dict()['class_weight']
class_weight

{0: 0.5,
 1: 0.75,
 2: 0.6666666666666667,
 3: 1.0,
 4: 0.6666666666666667,
 5: 0.03636363636363637,
 6: 0.13043478260869565,
 7: 0.3529411764705882,
 8: 0.75,
 9: 0.46153846153846156,
 10: 0.05454545454545454}

In [8]:
cat_cols_indices = np.where(X.dtypes != np.float)[0]
Xtrain = train.loc[:, train.columns != '80' ].copy()
Ytrain = train['80'].copy()-1
Xtest = test.loc[:, train.columns != '80' ].copy()

print("Xtrain: {} \nYtrain: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, Xtest.shape))

Xtrain: (403, 79) 
Ytrain: (403,) 
Xtest: (101, 79)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [9]:
FOLD = 7
NUM_SEED = 11
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], 11))
y_pred_final_cb = np.zeros((Xtest.shape[0], 11))
counter = 0
for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain.values)):
        counter += 1

        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]

        model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            class_weights=class_weight,
            learning_rate=0.07964,
            reg_lambda=0.95,
            bootstrap_type='Poisson',
            subsample=0.81,
            max_depth=7, 
            grow_policy='Lossguide',
            min_data_in_leaf=10, 
            max_leaves=152,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices,
                  early_stopping_rounds=200, verbose=100)

        y_pred = model.predict(val_x)
        y_pred_meta_cb[val] += model.predict_proba(val_x)
        y_pred_final_cb += model.predict_proba(Xtest)
        
        score = 100 * f1_score(val_y, y_pred, average='macro')
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=7.
  UserWarning,


0:	learn: 2.2535972	test: 2.3182072	best: 2.3182072 (0)	total: 99.5ms	remaining: 8m 17s
100:	learn: 0.1332744	test: 1.1055990	best: 1.1055990 (100)	total: 6.05s	remaining: 4m 53s
200:	learn: 0.0590360	test: 1.0495071	best: 1.0495071 (200)	total: 10.1s	remaining: 4m
300:	learn: 0.0370275	test: 1.0245049	best: 1.0239979 (299)	total: 16s	remaining: 4m 9s
400:	learn: 0.0268692	test: 1.0238455	best: 1.0227377 (393)	total: 21.1s	remaining: 4m 1s
500:	learn: 0.0207560	test: 1.0137698	best: 1.0131902 (493)	total: 24.1s	remaining: 3m 36s
600:	learn: 0.0170473	test: 1.0156984	best: 1.0124314 (546)	total: 25.4s	remaining: 3m 5s
700:	learn: 0.0144245	test: 1.0169227	best: 1.0124314 (546)	total: 26.7s	remaining: 2m 43s
bestTest = 1.01243139
bestIteration = 546
Shrink model to first 547 iterations.
Seed-24 | Fold-0 | OOF Score: 67.01523584640468
0:	learn: 2.2366239	test: 2.3118598	best: 2.3118598 (0)	total: 46.7ms	remaining: 3m 53s
100:	learn: 0.1266104	test: 0.8900012	best: 0.8887146 (98)	total: 4.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=7.
  UserWarning,


0:	learn: 2.2266540	test: 2.2839190	best: 2.2839190 (0)	total: 42.8ms	remaining: 3m 33s
100:	learn: 0.1278828	test: 1.4771660	best: 1.4388777 (64)	total: 2.54s	remaining: 2m 3s
200:	learn: 0.0566662	test: 1.5531685	best: 1.4388777 (64)	total: 3.91s	remaining: 1m 33s
bestTest = 1.43887766
bestIteration = 64
Shrink model to first 65 iterations.
Seed-3 | Fold-0 | OOF Score: 58.9349376114082
0:	learn: 2.2376047	test: 2.2923111	best: 2.2923111 (0)	total: 16.7ms	remaining: 1m 23s
100:	learn: 0.1300549	test: 0.7522644	best: 0.7522410 (99)	total: 1.42s	remaining: 1m 8s
200:	learn: 0.0555531	test: 0.6521999	best: 0.6520317 (199)	total: 2.79s	remaining: 1m 6s
300:	learn: 0.0349949	test: 0.6051628	best: 0.6051628 (300)	total: 4.11s	remaining: 1m 4s
400:	learn: 0.0253041	test: 0.5691818	best: 0.5691818 (400)	total: 5.5s	remaining: 1m 3s
500:	learn: 0.0196290	test: 0.5511899	best: 0.5511171 (499)	total: 6.83s	remaining: 1m 1s
600:	learn: 0.0160443	test: 0.5361564	best: 0.5361564 (600)	total: 8.13s	

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=7.
  UserWarning,


...

0:	learn: 2.2509603	test: 2.3074243	best: 2.3074243 (0)	total: 16.2ms	remaining: 1m 20s
100:	learn: 0.1306752	test: 0.9942676	best: 0.9942676 (100)	total: 1.42s	remaining: 1m 9s
200:	learn: 0.0591407	test: 0.8760753	best: 0.8760753 (200)	total: 2.76s	remaining: 1m 5s
300:	learn: 0.0372251	test: 0.8282529	best: 0.8264648 (296)	total: 4.05s	remaining: 1m 3s
400:	learn: 0.0269431	test: 0.7949671	best: 0.7949671 (400)	total: 5.3s	remaining: 1m
500:	learn: 0.0210452	test: 0.7679663	best: 0.7678504 (499)	total: 6.49s	remaining: 58.3s
600:	learn: 0.0174030	test: 0.7609185	best: 0.7609185 (600)	total: 7.72s	remaining: 56.5s
700:	learn: 0.0146609	test: 0.7495843	best: 0.7495843 (700)	total: 8.98s	remaining: 55.1s
800:	learn: 0.0126616	test: 0.7412576	best: 0.7411917 (799)	total: 10.2s	remaining: 53.5s
900:	learn: 0.0111679	test: 0.7330384	best: 0.7328880 (899)	total: 11.4s	remaining: 52.1s
1000:	learn: 0.0099824	test: 0.7264175	best: 0.7261778 (991)	total: 12.7s	remaining: 50.8s
1100:	learn: 0.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=7.
  UserWarning,


0:	learn: 2.2431368	test: 2.3202038	best: 2.3202038 (0)	total: 14.6ms	remaining: 1m 12s
100:	learn: 0.1297603	test: 1.2097101	best: 1.2097101 (100)	total: 1.46s	remaining: 1m 10s
200:	learn: 0.0584855	test: 1.1832990	best: 1.1750768 (166)	total: 2.9s	remaining: 1m 9s
300:	learn: 0.0371240	test: 1.1642746	best: 1.1623072 (295)	total: 4.25s	remaining: 1m 6s
400:	learn: 0.0267935	test: 1.1635393	best: 1.1603679 (359)	total: 5.53s	remaining: 1m 3s
500:	learn: 0.0209793	test: 1.1606868	best: 1.1597277 (483)	total: 6.88s	remaining: 1m 1s
600:	learn: 0.0171883	test: 1.1563518	best: 1.1561229 (596)	total: 8.16s	remaining: 59.8s
700:	learn: 0.0145367	test: 1.1604182	best: 1.1559367 (604)	total: 9.47s	remaining: 58.1s
800:	learn: 0.0125764	test: 1.1649881	best: 1.1559367 (604)	total: 10.7s	remaining: 56.4s
bestTest = 1.155936691
bestIteration = 604
Shrink model to first 605 iterations.
Seed-10 | Fold-0 | OOF Score: 70.74675324675324
0:	learn: 2.2202094	test: 2.3157474	best: 2.3157474 (0)	total: 

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=7.
  UserWarning,


0:	learn: 2.2629447	test: 2.3494888	best: 2.3494888 (0)	total: 13.5ms	remaining: 1m 7s
100:	learn: 0.1315423	test: 1.2182786	best: 1.2169529 (95)	total: 1.38s	remaining: 1m 6s
200:	learn: 0.0588929	test: 1.2521254	best: 1.2169529 (95)	total: 2.7s	remaining: 1m 4s
bestTest = 1.216952941
bestIteration = 95
Shrink model to first 96 iterations.
Seed-21 | Fold-0 | OOF Score: 63.24698029243484
0:	learn: 2.2403222	test: 2.2593103	best: 2.2593103 (0)	total: 13.5ms	remaining: 1m 7s
100:	learn: 0.1354483	test: 0.5821463	best: 0.5821463 (100)	total: 1.38s	remaining: 1m 6s
200:	learn: 0.0608963	test: 0.4903720	best: 0.4903720 (200)	total: 2.72s	remaining: 1m 4s
300:	learn: 0.0380591	test: 0.4625492	best: 0.4625492 (300)	total: 4s	remaining: 1m 2s
400:	learn: 0.0273825	test: 0.4481970	best: 0.4476616 (399)	total: 5.2s	remaining: 59.7s
500:	learn: 0.0215710	test: 0.4428423	best: 0.4424183 (497)	total: 6.39s	remaining: 57.4s
600:	learn: 0.0176478	test: 0.4369607	best: 0.4360838 (593)	total: 7.59s	rem

In [23]:
model.save_model("catboostmodel.cbm")